# A pretty report with a little wisdom

> A clever person solves a problem. A wise person avoids it.


In [ ]:
from comproaffitto.comproaffitto import a_function

a_function()

In [ ]:
# Define input parameters
params = {
    # Purchase scenario
    "purchase_price": 300000,  # House purchase price
    "down_payment_rate": 0.20,  # 20% down payment
    "mortgage_rate": 0.03,  # Annual mortgage rate
    "mortgage_years": 25,  # Mortgage duration
    "purchase_fees_rate": 0.04,  # Notary + taxes (simplified)
    "agency_fees_rate": 0.03,  # Agency fees for purchase
    "maintenance_rate": 0.015,  # Annual maintenance (1.5% of house value)
    # Rent scenario
    "monthly_rent": 1200,  # Base monthly rent
    "deposit_months": 3,  # Number of months for deposit
    "rent_agency_fees_months": 1,  # Agency fees in months of rent
    # Common expenses (monthly)
    "condo_fees": 150,  # Monthly condominium fees
    "utilities": 200,  # Monthly utilities estimate
    # Economic factors
    "inflation_rate": 0.02,  # Annual inflation rate
    "house_appreciation_rate": 0.01,  # Annual house value appreciation
    "investment_return_rate": 0.05,  # Return rate on invested capital
}